In [1]:
!pip install -U keras-tuner # for performing hyperparameter tuning
!pip install --upgrade tensorflow jax jaxlib # To ensure that we have the latest versions of TensorFlow and JAX installed.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [19]:
df = pd.read_csv('/content/Alphabets_data.csv')

In [20]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [21]:
df.drop(columns = ['letter'], inplace=True)
df

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [22]:
x = df.drop(columns = ['onpix'])
y = df['onpix']

In [23]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [26]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics =['accuracy'])

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1224 - loss: -1.5731
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1179 - loss: -1.5644
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1190 - loss: -1.5880
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1171 - loss: -1.6317
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1146 - loss: -1.6535
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1224 - loss: -1.6548
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1197 - loss: -1.6921
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1211 - loss: -1.7566
Epoch 9/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1173 - loss: -1.7708
Epoch 10/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1161 - loss: -1.8495
Epoch 11/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1235 - loss: -1.8236
Epoch 12/100
500/500 ━━━━━━━━━

In [27]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [28]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       156
           1       0.12      1.00      0.21       478
           2       0.00      0.00      0.00       860
           3       0.00      0.00      0.00       754
           4       0.00      0.00      0.00       643
           5       0.00      0.00      0.00       426
           6       0.00      0.00      0.00       285
           7       0.00      0.00      0.00       184
           8       0.00      0.00      0.00       112
           9       0.00      0.00      0.00        52
          10       0.00      0.00      0.00        24
          11       0.00      0.00      0.00        11
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         1

    accuracy                           0.12      4000
   macro avg       0.01   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
ann.evaluate(xtrain,ytrain)

500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1233 - loss: -5.0727


[-5.054544925689697, 0.1223749965429306]

#Hyperparameter tuning

In [30]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [39]:
def hyper(hp):
    model = Sequential()

    # Add input layer
    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=15)) # Changed input_dim to 15

    # Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))

    # Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [40]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

Reloading Tuner from ./untitled_project/tuner0.json


In [41]:
tuner.search(xtrain,ytrain,epochs = 3, validation_data = (xtest,ytest))
# search() is similar to fit()

Trial 3 Complete [00h 00m 15s]
val_accuracy: 0.11949999630451202

Best val_accuracy So Far: 0.11949999630451202
Total elapsed time: 00h 02m 38s


In [42]:
tuner.get_best_hyperparameters()[0].values # will give first model where accuracy is high

{'units0': 80,
 'activation0': 'tanh',
 'num_layers': 9,
 'optimizer': 'rmsprop',
 'units1': 72,
 'activation1': 'relu',
 'units2': 64,
 'activation2': 'tanh',
 'units3': 8,
 'activation3': 'sigmoid',
 'units4': 72,
 'activation4': 'relu',
 'units5': 104,
 'activation5': 'sigmoid',
 'units6': 24,
 'activation6': 'sigmoid',
 'units7': 96,
 'activation7': 'relu',
 'units8': 8,
 'activation8': 'relu'}

In [43]:
tuned_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1234 - loss: -4559.7158 - val_accuracy: 0.1195 - val_loss: -13267.3564
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1207 - loss: -18151.2793 - val_accuracy: 0.1195 - val_loss: -36811.7656
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1215 - loss: -45531.4336 - val_accuracy: 0.1195 - val_loss: -78384.0781
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1228 - loss: -92142.0703 - val_accuracy: 0.1195 - val_loss: -142737.9062
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.1251 - loss: -162134.2500 - val_accuracy: 0.1195 - val_loss: -235014.1094
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1213 - loss: -263241.2500 - val_accuracy: 0.1195 - val_loss: -360230.0312
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1206 - loss: -400306.5625 - val_accuracy: 0.1195 - val_loss: -523168.2812
Epoch 8/50
500/500 ━━━━━━━━━

In [45]:
tuned_model.evaluate(xtrain,ytrain) # Training accuracy

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1234 - loss: -112637496.0000


[-112432024.0, 0.1224374994635582]

In [46]:
tuned_model.evaluate(xtest,ytest)# Testing accuracy

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1181 - loss: -113912992.0000


[-113170424.0, 0.11949999630451202]